In [1]:
import os
from os import listdir
from os.path import join, dirname
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

PATH_REPO = dirname(os.getcwd())
PATH_DATA = join(PATH_REPO, 'data')

In [2]:
train = pd.read_csv(join(PATH_DATA, 'Train.csv'))

In [9]:
years = np.unique(train.year)
months = np.unique(train.month)

In [10]:
months

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [51]:
def format_data(data: pd.DataFrame) -> pd.DataFrame:

    data['date'] = data.apply(lambda x: datetime.strptime(str(x['year'])+'-'+str(x['month']), '%Y-%m'), axis=1)

    dates = np.unique(data.date)
    product_codes = np.unique(data.product_code)
    site_codes = np.unique(data.site_code)

    index = pd.MultiIndex.from_product([dates, product_codes, site_codes],
                                        names=['date', 'product_code', 'site_code'])
    
    data_all = pd.DataFrame(index=index).reset_index()

    data_formatted = pd.merge(data_all, data, how='left')


    return data_formatted

In [52]:
train_formatted = format_data(train)

In [ ]:
#https://forecastegy.com/posts/multiple-time-series-forecasting-with-xgboost-in-python/
#https://blogs.sap.com/2021/05/06/a-multivariate-time-series-modeling-and-forecasting-guide-with-python-machine-learning-client-for-sap-hana/

In [64]:
categorical_features = ['product_code', 'site_code']
for category in categorical_features:
    cat_values = np.unique(train_formatted.loc[:,category])
    for cat in cat_values:
        train_formatted[cat] = (train_formatted[category]==cat).astype(int)

In [69]:
train_v2 = train_formatted.loc[train_formatted.loc[:,'date']<'2019-01-01']
val = train_formatted.loc[train_formatted.loc[:,'date']>='2019-01-01']

In [73]:
from mlforecast import MLForecast
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
#from xgboost import XGBRegressor

from window_ops.rolling import rolling_mean, rolling_max, rolling_min

models = [make_pipeline(SimpleImputer(), 
                        LinearRegression())]


model = MLForecast(models=models,
                   freq='M',
                   lags=[1,2,3],
                   lag_transforms={
                       1: [(rolling_mean, 4), (rolling_min, 4), (rolling_max, 4)], # aplicado a uma janela W a partir do registro Lag
                   },
                   date_features=['date'],
                   num_threads=6)

In [92]:
train_v2['unique_id'] = train_v2['product_code'].astype(str) + '_' + train_v2['site_code'].astype(str)
train_v3 = train_v2.drop(['product_code', 'site_code', 'year', 'month', 'region', 'district'], axis=1)

/var/folders/jx/rrb06xvd6sd_k9800xyzvp4w0000gn/T/ipykernel_9065/2794549491.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_v2['unique_id'] = train_v2['product_code'].astype(str) + '_' + train_v2['site_code'].astype(str)
/var/folders/jx/rrb06xvd6sd_k9800xyzvp4w0000gn/T/ipykernel_9065/2794549491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_v2['unique_id'] = train_v2['product_code'].astype(str) + '_' + train_v2['site_code'].astype(str)


In [93]:
train_v3

,date,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,AS17005,...,C5016,C5017,C5018,C5019,C5020,C5021,C5060,C5063,C5066,unique_id
0,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,AS17005_C1004
1,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,AS17005_C1007
2,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,AS17005_C1008
3,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,AS17005_C1009
4,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,AS17005_C1010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61375,2018-12-01,144.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,AS46000_C5020
61376,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,1,0,0,0,AS46000_C5021
61377,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,1,0,0,AS46000_C5060
61378,2018-12-01,89.0,0.0,2.0,0.0,87.0,2.0,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,AS46000_C5063


In [78]:
static_features = train_v3.columns[9:]

In [94]:
train_v3['stock_distributed'] = train_v3['stock_distributed'].fillna(0)

In [97]:
model.fit(train_v3, id_col='unique_id',time_col='date', target_col='stock_distributed', static_features=static_features, max_horizon=3)


/Users/alejandrobonell/.pyenv/versions/miniforge3-4.10.3-10/envs/ts_contraceptives/lib/python3.9/site-packages/mlforecast/core.py:333: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat] = self.features_[feat][self.restore_idxs]
/Users/alejandrobonell/.pyenv/versions/miniforge3-4.10.3-10/envs/ts_contraceptives/lib/python3.9/site-packages/mlforecast/core.py:333: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat] = self.features_[feat][self.restore_idxs]
/Users/alejandrobonell/.pyenv/versions/miniforge3-4.10.3-10/envs/ts_co

MLForecast(models=[Pipeline], freq=<Week: weekday=6>, lag_features=['lag1', 'lag2', 'lag4', 'rolling_mean_lag1_window_size4', 'rolling_min_lag1_window_size4', 'rolling_max_lag1_window_size4'], date_features=['week', 'month'], num_threads=6)

In [96]:
p = model.predict(horizon=3, dynamic_dfs=[valid[['unique_id','ds']+dynamic_features]])


ValueError: Data doesn't contain unique_id column

In [76]:
train_v3

,date,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,AS17005,...,C5015,C5016,C5017,C5018,C5019,C5020,C5021,C5060,C5063,C5066
0,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0
3,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0
4,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61375,2018-12-01,144.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0,...,0,0,0,0,0,1,0,0,0,0
61376,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,1,0,0,0
61377,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,1,0,0
61378,2018-12-01,89.0,0.0,2.0,0.0,87.0,2.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,1,0


In [49]:
train['date'] = train.apply(lambda x: datetime.strptime(str(x['year'])+'-'+str(x['month']), '%Y-%m'), axis=1)

In [45]:
from datetime import datetime

In [57]:
product_c = 'AS17005'
site_c = 'C1004'

train_formatted.loc[(train_formatted.loc[:,'product_code']==product_c) & (train_formatted.loc[:,'site_code']==site_c),'stock_distributed'].isna().sum()

42

,date,product_code,site_code,year,month,region,district,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered
300,2016-01-01,AS21126,C5015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,2016-01-01,AS21126,C5016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,2016-01-01,AS21126,C5017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,2016-01-01,AS21126,C5018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,2016-01-01,AS21126,C5019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,2016-01-01,AS21126,C5020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,2016-01-01,AS21126,C5021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,2016-01-01,AS21126,C5060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,2016-01-01,AS21126,C5063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,2016-01-01,AS21126,C5066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
